# 1. Import and Install Dependencies

In [5]:
pip install mediapipe keras tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/241.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/241.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/241.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/241.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/241.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/241.6 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/241.6 kB 187.9 kB/s eta 0:00:02
   ---- ---------------------------------- 30.7/241.6 kB 187.9 kB/s eta 0:00:02
   ---- ---------------------------------- 30.7/241.6 kB 187.9 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/241.6 kB 131.3 kB/s eta 0:00:02
   ------ -------------------------------- 41.0/241.6 kB 131.3 kB/s eta 0:00:02
   ----------- --------------------------- 71.7/241.6 kB 187.3 kB/s eta 0:00:01
   ----------- ----


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import numpy as np
import os
import time
import datetime
import mediapipe as mp
import datetime
from keras.models import load_model
from keras.layers import Input, Dense
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Reshape, LSTM, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
#%matplotlib inline 
import copy
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras import models
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.layers import Dropout
from sklearn.metrics import confusion_matrix



ModuleNotFoundError: No module named 'keras.utils.vis_utils'

# 2. Plot Keypoints using MediaPipe Holistic AI

In [3]:
def DrawText(img,sText,x,y):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    posf = (x,y)
    fontScale              = 5
    fontColor              = (255,255,255)
    thickness              = 1
    lineType               = 2
    print("Masuk")
    cv2.putText(img,sText, 
        posf, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType)
    return copy.deepcopy(img)
    

2.1. Initialize New File

In [4]:
def GetFileName():
        x = datetime.datetime.now()
        s = x.strftime('%Y-%m-%d-%H%M%S%f')
        return s

2.2. Create a New Directory to store File

In [5]:
def CreateDir(path):
    ls = [];
    head_tail = os.path.split(path)
    ls.append(path)
    while len(head_tail[1])>0:
        head_tail = os.path.split(path)
        path = head_tail[0]
        ls.append(path)
        head_tail = os.path.split(path)   
    for i in range(len(ls)-2,-1,-1):
        sf =ls[i]
        isExist = os.path.exists(sf)
        if not isExist:
            os.makedirs(sf)
NamaDataSet = "Kata"

# 3. Draw Landmarks / Create Skeletal Frame

In [6]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(bimage, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections


In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                            # mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                            # mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            # ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 

# 4. Create Dataset from LIVE Camera Recording

In [8]:
def Dataset(NoKamera,NamaDataSet):
    DirektoriData = "A:\\buat anaconda\\dataimage"+"\\"+NamaDataSet
    #+"\\"+GetFileName()    
    CreateDir(DirektoriData)        
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_holistic = mp.solutions.holistic
    imsize=(640, 480)
    TimeStart = time.time() 
    TimeNow = time.time() +10
    FrameRate = 10
    
    
    cap = cv2.VideoCapture(NoKamera,cv2.CAP_DSHOW)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      
      while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue
    

        image.flags.writeable = False                  # Image is no longer writeable
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, imsize)
        results = holistic.process(image)                 # Make prediction
        
        image_height, image_width, _ = image.shape
        
        coords = []
        coordsy = []
      
        if not results.pose_landmarks:
            continue
            
        for A in results.pose_landmarks.landmark:
            
            if (A.x>0.01)and(A.x<1-0.01)and(A.y>0.01)and(A.y<1-0.01):
                cx, cy = A.x * image_width, A.y*image_height 
                coords.append(cx) 
                coordsy.append(cy)
        if len(coords) ==0:
            continue
        elif len(coordsy) ==0:
            continue

        #if results.pose_landmarks:
        #    pose_landmarks = results.pose_landmarks.landmark
        #    for A in pose_landmarks:
        #      cx, cy = A.x * image_width, A.y*image_height                                                                                                                                                                                                                                                                                                                                                                                            
        #      coords.append(cx) 
        #      coordsy.append(cy) 

        x_max = max(coords)
        y_max = max(coordsy)
        x_min = min(coords)
        y_min = min(coordsy)
            

        cv2.rectangle(image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 255, 0), 2)

        if (int (x_max) < 0):
            x_max = 1
        elif (int (x_min) < 0):
            x_min = 1
        elif (int (x_max) < 0):
            y_max = 1
        elif (int (x_min) < 0):
            y_min = 1   
            
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        bimage = np.zeros((image_height,image_width,3), np.uint8)
        cv2.rectangle(bimage,(int(x_min), int(y_min)),(int(x_max), int(y_max)),(0,255,0),2) #(KALAU MAU PAKAI BOUNDING BOX)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
        mp_drawing.draw_landmarks(bimage, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
      
        
 
              

        image = cv2.rectangle(image,(int(x_min), int(y_min)),(int(x_max), int(y_max)),(255,255,0),2)
        
 
        cropped_image = bimage[(int(y_min)):(int(y_max)), (int(x_min)):(int(x_max)),:]
        dy =y_max -y_min
        dx = x_max -x_min
        print(dy,dx)
        print(cropped_image.shape) 
        TimeNow = time.time() 
        if TimeNow-TimeStart>1/FrameRate:
            print(cropped_image.shape)
            TimeStart = TimeNow
            sFile = DirektoriData+"\\"+GetFileName()
            imsize2=(128,128)
            cropped_image = cv2.resize(cropped_image, imsize2)
            #cropped_image = cv2.resize(bimage, imsize2)
            cv2.imwrite(sFile+'.jpg', cropped_image)
            #cv2.imwrite(sFile+'.png', image)
        
        cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        if cv2.waitKey(10) & 0xFF == ord('q'):
              break
    cap.release()
    cv2.destroyAllWindows()

# 5. Predict Pose based on Video Frame

In [9]:
def PredictPose(NoKamera,LabelKelas):
    ModelCNN = load_model('weight3_test.h5') 
    

    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_holistic = mp.solutions.holistic
    imsize=(640, 480)

    # For webcam input:
    cap = cv2.VideoCapture(NoKamera,cv2.CAP_DSHOW)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
      
      while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue
    

        image.flags.writeable = False                  # Image is no longer writeable
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, imsize)
        results = holistic.process(image)                 # Make prediction
        
        height, width, _ = image.shape
        coords = []
        coordsy = []
                                                                                                                                  

        if not results.pose_landmarks:
                continue
        for A in results.pose_landmarks.landmark:

            if (A.x>0.01)and(A.x<1-0.01)and(A.y>0.01)and(A.y<1-0.01):
                cx, cy = A.x * width, A.y*height 
                coords.append(cx) 
                coordsy.append(cy)
        if len(coords) ==0:
            continue
        if len(coordsy) ==0:
            continue

            #if results.pose_landmarks:
            #    pose_landmarks = results.pose_landmarks.landmark
            #    for A in pose_landmarks:
            #      cx, cy = A.x * image_width, A.y*image_height                                                                                                                                                                                                                                                                                                                                                                                            
            #      coords.append(cx) 
            #      coordsy.append(cy) 

        x_max = max(coords)
        y_max = max(coordsy)
        x_min = min(coords)
        y_min = min(coordsy)
        cv2.rectangle(image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 255, 0), 2)
        
        if (int (x_max) < 0):
            x_max = 1
        elif (int (x_min) < 0):
            x_min = 1
        elif (int (y_max) < 0):
            y_max = 1
        elif (int (y_min) < 0):
            y_min = 1   
        
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        bimage = np.zeros((height,width,3), np.uint8)
        #cv2.rectangle(bimage,(int(x_min), int(y_min)),(int(x_max), int(y_max)),(0,255,0),2)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
        mp_drawing.draw_landmarks(bimage, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
       
        
         
              
        cropped_image = image[(int(y_min)):(int(y_max)), (int(x_min)):(int(x_max)),:]
        idx = Klasifikasi(bimage, ModelCNN)
        x=60
        y=60
        image= cv2.flip(image, 1)
    
        if idx>=0:
            cv2.putText(image,LabelKelas[idx], (x,y), cv2.FONT_HERSHEY_SIMPLEX,2.0, (255, 255, 0), 3)
        

        cv2.imshow('MediaPipe Pose', image)
                
       
        if cv2.waitKey(5) & 0xFF == ord('q'):
              break
    cap.release()
    cv2.destroyAllWindows()

# 6. Classification of Dataset 

In [10]:
def Klasifikasi(Image,ModelCNN):

  X=[]
  ls = [];

  img= copy.deepcopy(Image)
  img=cv2.resize(img,(128,128))
  img= np.asarray(img)/255
  img=img.astype('float32')
  X.append(img)  
  X=np.array(X)
  X=X.astype('float32')
  hs=ModelCNN.predict(X,verbose=0)

  if hs.max()>0.5:
      idx = np.max(np.where( hs == hs.max()))
  else:
    idx=-1
      
 
  return idx

 # 7. Create a Convolutional Neural Network Model

In [11]:
def ModelCNN(JumlahKelas):
    input_img = Input(shape=(128, 128, 3)) 

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)  
    x = MaxPooling2D((2, 2), padding='same')(x)   
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)   
    x = MaxPooling2D((2, 2), padding='same')(x)   
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    #x = Reshape((-1, 32))(x)
    #x = LSTM(64, return_sequences=True, activation='relu')(x)
    #x = LSTM(128, return_sequences=True, activation='relu')(x)
    #x = LSTM(64, return_sequences=False, activation='relu')(x)
    x = Dense(100,activation='relu')(x)
    #x = Dense(50,activation='relu')(x)
    x = Dense(50,activation='sigmoid')(x)
    x = Dense(JumlahKelas,activation='softmax')(x)
    ModelCNN = Model(input_img, x)  
    ModelCNN.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    #ModelCNN.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    #sparse_categorical_crossentropy
    #categorical_crossentropy
    #plot_model(ModelCNN, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    return ModelCNN


 # 7. Load Citra Training Data (Object Classes)

In [12]:
def LoadCitraTraining(sDir,LabelKelas):
  
  JumlahKelas=len(LabelKelas)
  TargetKelas = np.eye(JumlahKelas)
  
  # Menyiapkan variabel list untuk data menampung citra dan data target
  X=[]#Menampung Data Citra
  T=[]#Menampung Target
  for i in range(len(LabelKelas)):    
    #Membaca file citra di setiap direktori data set  
    DirKelas = os.path.join(sDir, LabelKelas[i])
    files = os.listdir(DirKelas)
    
    for f in files:
      ff=f.lower()  
      print(f)
      #memilih citra dengan extensi jpg,jpeg,dan png
      if (ff.endswith('.jpg')):
         NmFile = os.path.join(DirKelas,f)
         #membaca citra berwarna sebagai data bertipe double 
         img= np.double(cv2.imread(NmFile,1))
         img=cv2.resize(img,(128,128));
         #Normalisasi data citra menjadi sehingga maksimum menjadi 1
         img= np.asarray(img)/255;
         img=img.astype('float32')
         #Menambahkan citra dan target ke daftar
         X.append(img)
         T.append(TargetKelas[i])
     #--------akhir loop :Pfor f in files-----------------
  #-----akhir  loop :for i in range(len(LabelKelas))----
  
  #Mengubah List Menjadi numppy array
  X=np.array(X)
  T=np.array(T)
  X=X.astype('float32')
  T=T.astype('float32')
  return X,T


7.1. Train the CNN to Recognize => Classify Data

In [13]:
def TrainCNNLSTM(DirektoriDataSet,LabelKelas,NamaFileBobot ='weight3_test.h5' ):
    #tf.config.experimental_run_functions_eagerly(True)
    #tf.data.experimental.enable_debug_mode()
    #Membaca Data training dan label Kelas 
    X,D=LoadCitraTraining(DirektoriDataSet,LabelKelas)
    JumlahKelas = len(LabelKelas)
    #Membuat Model CNN
    ModelCNNlstm =ModelCNN(JumlahKelas)
    #Trainng
    history = ModelCNNlstm.fit(X, D,epochs=20,validation_split=0.1)
    #validation_split = 0.5
    #Menyimpan hasil learning
    ModelCNNlstm.save(NamaFileBobot)
    #Mengembalikan output 
    return ModelCNNlstm,history

7.2. Make Predictions for the Pose based on Primary Dataset

In [14]:
def TesPosePrediction(DirDataSet,DirKlasifikasi,LabelKelas,ModelCNN=[]):
#Apabila parameter input ModelCNN tidak di isi maka
#   akan menggunakan bobot pada file 'weight.h5
  if not(ModelCNN):
      ModelCNN = load_model('weight3_test.h5') 
      
#Menyiapkan Data input Yang akan di kasifikasikan
  X=[]
  ls = [];
  DirKelas = DirDataSet+"\\"+DirKlasifikasi
  print(DirKelas)
  files = os.listdir(DirKelas)
  n=0;
  for f in files:
      ff=f.lower()  
      print(f)
      if (ff.endswith('.jpg')):
         ls.append(ff) 
         NmFile = os.path.join(DirKelas,f)
         img= cv2.imread(NmFile,1)
         img=cv2.resize(img,(128,128))
         img= np.asarray(img)/255
         img=img.astype('float32')
         X.append(img)
     #----Akhir if-------------
  #---Akhir For 
  X=np.array(X)
  X=X.astype('float32')
  #Melakukan prediksi Klasifikasi
  hs=ModelCNN.predict(X)
  
  LKlasifikasi=[];
  LKelasCitra =[];
  n = X.shape[0]
  for i in range(n):
      v=hs[i,:]
      if v.max()>0.5:
          idx = np.max(np.where( v == v.max()))
          LKelasCitra.append(LabelKelas[idx])
      else:
          idx=-1
          LKelasCitra.append("-")
      #------akhir if
      LKlasifikasi.append(idx);
  #----akhir for
  LKlasifikasi = np.array(LKlasifikasi)
  return ls, hs, LKelasCitra

7.3. Collect Dataset and Store to Assigned Directory

In [ ]:
# mengambil data set dan menyimpan ke folder
Dataset(0,"Paman")

In [ ]:
#Training Data Set
DirektoriDataSet="D:\\temp\\trainingdataset4"
#   Data Set disimpan dalam direktori yang sama dengan nama kelas    

#b. Label Data Set 
LabelKelas=("A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S")
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

#c. Inisialisasi parameter Training
#JumlahEpoh = 100;

#d. training
ModelCNN,history = TrainCNNLSTM(DirektoriDataSet,LabelKelas )
ModelCNN.summary()

#c. Menampilkan Grafik Loss dan accuracy
plt.plot(history.history['loss'])

plt.title('model loss')
plt.ylabel('loss/accuracy')
plt.xlabel('epoch')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy')
plt.title('Model Accuracy')
plt.xlabel('epoch')
plt.ylabel('categorical_accuracy')
plt.legend(['train','test'])
plt.show()


7.4. Predict Pose Test based on Classified Data

In [15]:
LabelKelas=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"]

PredictPose(0,LabelKelas)


In [ ]:
LabelKelas=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S"]
DirektoriDataSet="D:\\temp\\trainingdataset4"

DirKlasifikasi="C"
Res = TesPosePrediction(DirektoriDataSet,DirKlasifikasi,LabelKelas)
print(Res[0])
print(Res[1])

# 8. Create a Confusion Matrix to Assess Overall Performance

In [ ]:
# Assuming you have the predicted labels and true labels for test data
X_test, D_test = LoadCitraTraining(DirektoriDataSet, LabelKelas)

# Predict labels for the test data
predicted_labels = ModelCNN.predict(X_test)
predicted_labels = np.argmax(predicted_labels, axis=1)  # Convert predicted probabilities to class labels

# Convert one-hot encoded true labels to class labels
true_labels = np.argmax(D_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(LabelKelas))
plt.xticks(tick_marks, LabelKelas, rotation=45)
plt.yticks(tick_marks, LabelKelas)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 9. Determine Result Accuracy & Precision, Correction, Error

In [ ]:
# Assuming you have the predicted labels and true labels for test data
X_test, D_test = LoadCitraTraining(DirektoriDataSet, LabelKelas)

# Load or initialize your trained model
model = load_model('weight3_test.h5')  # Replace 'path_to_your_model.h5' with the actual path to your model

# Predict labels for the test data
predicted_labels = model.predict(X_test)
predicted_labels = np.argmax(predicted_labels, axis=1)  # Convert predicted probabilities to class labels

# Convert one-hot encoded true labels to class labels
true_labels = np.argmax(D_test, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Calculate precision for each class
precisions = precision_score(true_labels, predicted_labels, average=None)

# Calculate recall for each class
recalls = recall_score(true_labels, predicted_labels, average=None)

# Calculate F1-score for each class
f1_scores = f1_score(true_labels, predicted_labels, average=None)

# Print the precision, recall, and F1-score table
print("Class\t\tPrecision\t1-Precision\tRecall\t\t1-Recall\tF1-score")
for i in range(len(LabelKelas)):
    print("{}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(LabelKelas[i], precisions[i], 1-precisions[i],
                                                                         recalls[i], 1-recalls[i], f1_scores[i]))